# Gerando quantitativos

Neste notebook serão gerados os seuintes quantitativos para os 14 CEUs selecionados:

* Área total
* Perímetro
* Área projeção edificada
* Área vegetada
* Volume edificado
* Taxa de ocupação
* Coeficiente volumétrico de aproveitamento


In [115]:
import geopandas as gpd
import pandas as pd

In [84]:
gdf_ceus = gpd.read_file('resultados/ceus-escolhidos.geojson')

In [85]:
gdf_veg = gpd.read_file('resultados/cobertura_vegetal_CEUs_recortado.gpkg')

In [105]:
gdf_edif = gpd.read_file('resultados/edifícios.gpkg')
gdf_edif.crs = 'EPSG:31983'

In [106]:
gdf_ceus = gdf_ceus[['cd_identificador', 'nm_equipamento', 'geometry']]

In [107]:
gdf_veg = gdf_veg[['tx_descricao_categoria_subcategoria', 'geometry']]

In [108]:
gdf_edif = gdf_edif[['altura_int', 'geometry']]

In [109]:
gdf_edif.loc[:, 'volume_edificado'] = gdf_edif.area * gdf_edif.altura_int

In [110]:
gdf_edif.loc[:, 'area_projecao_edificada'] = gdf_edif.area

In [124]:
edif = (gpd.sjoin(gdf_ceus, gdf_edif, how='left', op='contains')
        .rename_axis('id')
        .groupby('id')
        .sum()[['volume_edificado', 'area_projecao_edificada']])

In [134]:
gdf_ceus = gdf_ceus.join(edif)

In [137]:
gdf_veg.loc[:, 'area_vegetada'] = gdf_veg.area

In [139]:
veg = (gpd.sjoin(gdf_ceus, gdf_veg, how='left', op='contains')
        .rename_axis('id')
        .groupby('id')
        .sum()[['area_vegetada']])

In [141]:
gdf_ceus = gdf_ceus.join(veg)

In [144]:
gdf_ceus.loc[:, 'area_total'] = gdf_ceus.area

In [149]:
gdf_ceus.loc[:, 'perimetro'] = gdf_ceus.length

In [151]:
gdf_ceus.loc[:, 'taxa_de_ocupacao'] = gdf_ceus.area_projecao_edificada / gdf_ceus.area_total

In [152]:
gdf_ceus.loc[:, 'coeficiente_volumetrico_aproveitamento'] = gdf_ceus.volume_edificado / gdf_ceus.area_total

In [153]:
gdf_ceus

,cd_identificador,nm_equipamento,geometry,volume_edificado,area_projecao_edificada,area_vegetada,area_total,perimetro,taxa_de_ocupacao,coeficiente_volumetrico_aproveitamento
0,50102.0,CEU CEI UIRAPURU,"POLYGON ((317434.250 7388857.309, 317347.376 7...",75849.120000,6494.56,1231.376962,20393.378568,823.891599,0.318464,3.719301
1,50242.0,CEU CEI CAMPO LIMPO,"POLYGON ((318559.041 7384884.278, 318563.964 7...",68787.440000,5701.76,87.031043,32623.638982,891.119086,0.174774,2.108515
2,50415.0,CEU CEI GUARAPIRANGA,"POLYGON ((320219.934 7378087.306, 320225.207 7...",102852.400000,10332.96,6773.126496,77070.098593,1242.308071,0.134072,1.334531
3,50649.0,CEU CEI PERUS,"POLYGON ((320904.370 7410458.125, 320943.634 7...",64403.360000,4900.08,158.344544,20488.761606,1022.427347,0.239159,3.143351
4,51660.0,CEU CEI CIDADE DUTRA,"POLYGON ((326271.898 7376707.519, 326265.685 7...",65765.200000,5053.04,18.871165,12043.412306,529.278391,0.419569,5.460678
5,52295.0,CEU CEI TRES LAGOS,"POLYGON ((328814.200 7370348.695, 328806.100 7...",75599.120000,5669.76,6360.202979,47917.349302,925.582953,0.118324,1.577698
6,53337.0,CEU CEI CAMINHO DO MAR,"POLYGON ((333388.003 7381564.589, 333408.163 7...",99841.680000,5779.28,1217.812367,42249.994437,1104.578325,0.136788,2.363117
7,54200.0,CEU CEI MENINOS,"POLYGON ((338546.576 7386740.311, 338541.382 7...",64987.920000,4986.96,430.185286,33943.161764,789.297742,0.146921,1.914610
8,54508.0,CEU CEI WALTER DE ANDRADE PROF,"POLYGON ((341448.825 7398388.873, 341451.850 7...",105997.840001,9980.48,1498.829315,29859.065336,649.290816,0.334253,3.549938
9,55098.0,CEU CEI QUINTA DO SOL,"POLYGON ((347223.688 7400518.334, 347235.324 7...",82764.000001,5048.08,41.404250,9930.129737,464.577143,0.508360,8.334634


In [155]:
gdf_ceus.to_file('resultados/quantitativos.gpkg', driver='GPKG')